Reference: [Introduction To Conformal Prediction With Python: A Short Guide For Quantifying Uncertainty Of Machine Learning Models](https://www.amazon.com/dp/B0BW2X919P)

# import modules

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from mapie.classification import MapieClassifier
from mapie.metrics import classification_coverage_score
from mapie.metrics import classification_mean_width_score

# datasets processing

In [2]:
# Read in the data from Excel file
bean_data_file = "datasets/DryBeanDataset/Dry_Bean_Dataset.xlsx"
beans = pd.read_excel(bean_data_file)

# Labels are characters but should be integers for sklearn
le = LabelEncoder()
beans["Class"] = le.fit_transform(beans["Class"])
# Split data into classification target and features
X, y = beans.drop("Class", axis=1), beans["Class"]

# Split of training data
X_train, X_rest1, y_train, y_rest1 = train_test_split(X, y, train_size=10000, random_state=2)

# From the remaining data, split of test data
X_test, X_rest2, y_test, y_rest2 = train_test_split( X_rest1, y_rest1, train_size=1000, random_state=42)

# Split remaining into calibration and "new" data
X_calib, X_new, y_calib, y_new = train_test_split(X_rest2, y_rest2, train_size=1000, random_state=42)

# Fit the model
model = GaussianNB().fit(X_train, y_train)

# 7.2 The naive method doesn’t work

The naive approach would be to take the probabilities at face value and assume they are well calibrated. So, to generate
a prediction set with at least $ 1 - \alpha $ probability, we take the following naive approach: For each data instance,
we add up the “probabilities” until the cumulative score of $ 1 - \alpha $ is exceeded, starting with the highest.

In [3]:
# naive method
# Initialize the MapieClassifier
mapie_score = MapieClassifier(model, cv="prefit", method="naive")
# Calibration step
mapie_score.fit(X_train, y_train)
# Prediction step
y_pred, y_set = mapie_score.predict(X_new, alpha=0.05)
# Removing the alpha-dimension
y_set = np.squeeze(y_set)

Coverage: 90.32%
Avg. set size: 1.41


<font color=red> Note The naive method does not rely on calibration data. So it makes no difference what data you use in
mapie_score.fit(). But if you skip this step, you will get an error from MAPIE. </font>

Now it’s time to evaluate the method. We will look at the average size of the prediction set, where the smaller the
better, as long as the coverage is guaranteed. We will also check the coverage of the prediction sets that we generated
for our dataset $X_{new}$, for which we have the labels $y_{new}$.

Fortunately, MAPIE already implements functions to compute coverage and set sizes, so we can quickly assess the
performance of the naive method.

In [ ]:
cov = classification_coverage_score(y_new, y_set)
setsize = classification_mean_width_score(y_set)
print('Coverage: {:.2%}'.format(cov))
print('Avg. set size: {:.2f}'.format(setsize))

As expected, the coverage is too low because the scores in the training data are not suitable for finding the threshold
$ \hat{q}$. The model is too overconfident and therefore the coverage is too low for new data.

# 7.3 Score method in MAPIE

## 7.3.1 How it works
The score method follows the standard conformal prediction recipe. We begin with the training and calibration steps:
- Split the data into training and calibration (size $n_{cal}$)
- Train the model on the training data
- Compute non-conformity scores
- Compute the quantile level: $q_{level} = 1 - ceil \left(\left(n_{cal}+1 \right)\left(1-\alpha \right)\right)/n_{cal}$
- Compute the $q_{level}$ quantile $\hat{q}$ of the scores $s_i$.

The quantile level is $1 - \alpha$ multiplied by $n_{cal} + 1 / n_{cal}$, which serves as a finite sample correction
term. The term is only relevant when the calibration dataset is small: If $\alpha=0.05$ and $n_{cal}=50$,
then $q_{level} = 0.97$. If $n_{cal} = 1000$, then $q_{level} = 0.951$.

Now we can use this threshold $\hat{q}$ to create prediction sets for new data:
- Get a new sample $x_new$
- Compute $s(y, x_{new})$ for all classes $y$
- Pick all classes $y$ where $s(y, x_{new}) \le \hat{q}$

Now we can enjoy our prediction sets with the following marginal coverage guarantee:
$$
1-\alpha \le \mathbb{P} \left( Y_{new} \in C\left( X_{new} \right) \right) \le 1 - \alpha + \frac{1}{n_{cal} + 1}
$$
Note that there is not only a lower bound, but also an upper bound. The larger the calibration set, the tighter the upper bound.

<font color=red> Warning: Don’t use the score method if you need the prediction sets to be adaptive. </font>

In [4]:
# In MAPIE we can use the score method by selecting method='score'
mapie_score = MapieClassifier(model, cv='prefit', method='score')
mapie_score.fit(X_calib, y_calib)
y_pred, y_set = mapie_score.predict(X_new, alpha=0.05)
y_set = np.squeeze(y_set)

# analyze the coverage and sizes of the prediction sets.
cov = classification_coverage_score(y_new, y_set)
setsize = classification_mean_width_score(y_set)
print('Coverage: {:.2%}'.format(cov))
print("Avg. set size: {:.2f}".format(setsize))

Coverage: 96.28%
Avg. set size: 1.83


The coverage is now good, given $ \alpha = 0.05 $, which implies a 95% coverage for the prediction sets. The average set
size is also quite small, which is good. But all is not well when using the score method.

##  The score method lacks adaptivity
While all conformal prediction methods guarantee marginal coverage, the score method isn’t adaptive to the difficulty of
each classification.

### Marginal Coverage
Marginal coverage means that on average $ 1 - \alpha $ of the predicted regions contain the true label for new data
samples. All conformal predictors guarantee marginal coverage. Marginal coverage can be estimated as the percentage of
prediction sets that cover the true class for a new dataset (that is exchangeable with the calibration set).

But we have no guarantee that the coverage is $ 1 - \alpha $ for every data point, not even for groups in the data, such
as for every class. Perhaps one type of bean is much harder to classify than another. Then it might well be that the
coverage for one class is higher than $ 1 - \alpha $ and for the other class it’s lower, but on average the coverage is
$ 1 - \alpha $ (= marginal coverage achieved).

If you want the conformal predictor to also achieve the coverage guarantee for groups of the data, or even for any point
in the feature space, then we speak of conditional coverage.

## Conditional Coverage
Conditional coverage means that the coverage of $ 1 - \alpha $ is not only true on average, but also conditional on some
feature or grouping of the data. Perfect conditional coverage (broken down to each data point) can’t be guaranteed, only
attempted. Conditional coverage for defined groups is possible.

Class-conditional coverage is critical for the fictional use case of selling beans with a guarantee. Marginal coverage
isn’t sufficient for this use case, since it is possible that one or more varieties are below marginal coverage
(and therefore others are above it).

In [5]:
def class_wise_performance(y_new, y_set, classes):
    df = pd.DataFrame()
    # Loop through the classes
    for i in range(len(classes)):
        # Calculate the coverage and set size for the current class
        ynew = y_new.values[y_new.values == i]
        yscore = y_set[y_new.values == i]
        cov = classification_coverage_score(ynew, yscore)
        size = classification_mean_width_score(yscore)

        # Create a new dataframe with the calculated values
        temp_df = pd.DataFrame({"class": [classes[i]], "coverage": [cov], "avg. set size": [size] }, index = [i])
        # Concatenate the new dataframe with the existing one
        df = pd.concat([df, temp_df])

    return df

print(class_wise_performance(y_new, y_set, le.classes_))

      class  coverage  avg. set size
0  BARBUNYA  0.925287       2.137931
1    BOMBAY  1.000000       1.000000
2      CALI  0.965909       2.130682
3  DERMASON  0.972906       1.463054
4     HOROZ  0.938525       1.872951
5     SEKER  0.970213       2.017021
6      SIRA  0.974277       1.974277


## Conclusion
The score method is easy to implement and understand. It provides both lower and upper bounds on coverage and produces
the smallest average prediction sets compared to other conformal classification methods. However, the method is not
adaptive: the coverage guarantee only holds on average. Diﬀicult classification cases may have prediction sets that are
too small, and easy cases may have sets that are too large.

## Better Recommendation
It’s better to use options that are designed to be adaptive. There are several ways to get adaptive prediction sets in
classification, and we will learn about all of them in the following sections.
- APS and RAPS are adaptive methods
- Group-balanced conformal prediction guarantees coverage for groups in the data
- Class-conditional conformal prediction guarantees coverage for all classes

# Use Adaptive Prediction Sets (APS) for conditional coverage

Before we dive into the theory behind APS, let’s talk more about adaptivity.

Imagine you have two clusters in your data. Half of your data is easy to classify, the other half is difficult.
A conformal predictor can achieve 90% coverage through the following scenario: The coverage for the prediction sets of
the easy data points is 100% and the coverage for the prediction sets of the difficult data points is 80%. The conformal
predictor, in this case, is not adaptive. In practice, this means that the prediction sets for the easy data are too
large on average, and the prediction sets for the difficult data are too small.

Adaptivity: A conformal prediction algorithm is adaptive if it not only achieves marginal coverage, but also
(approximately) conditional coverage.

## How APS work

Let’s remember the score method: The score method uses only the probability of the true label and ignores all other
probabilities. For a cat image with probabilities cat=0.3, lion=0.6, and dog=0.1, the non-conformity score $s_i$ would
be $s_i = 1 − 0.3 = 0.7$, since we only use the probability of the cat class. APS uses a different non-conformity score:
The idea is to add up the probabilities, starting with the largest, down to the true class. So for the example above,
the score would be 0.6 + 0.3 = 0.9.

Calibration works as usual:
- Compute all cumulative scores $s_i$ for the calibration data
- Compute the quantile level $q_{level} = 1 - ceil \left(\left(n_{cal}+1 \right) \alpha \right)/n_{cal}$
- Compute the $q_{level}$ quantile $\hat{q}$ for the calibration data so that (roughly) $1-\alpha$ of the scores are below $\hat{q}$

The prediction step also follows the usual recipe:
- Compute all class probabilities for a new data point $x_{new}$
- Sort the class probabilities in descending order
- Include classes, starting with the largest probability, until the threshold $\hat{q}$ is reached

The last step is not well-defined: Do we stop at the class just below the threshold or above it? We have three options
regarding the inclusion in the prediction set:
- Include the label above the threshold
- Don’t include the label above the threshold
- Include the label at random

# APS in MAPIE

All three inclusion options are implemented in MAPIE. They all start with the same initialization and calibration process.

In [6]:
# For the randomized option, the classifier draws random numbers
# So to make the code reproducible, we have to set the random_state
mapie_score = MapieClassifier(model, cv="prefit", method="cumulated_score", random_state=1)
mapie_score.fit(X_calib, y_calib)

MapieClassifier(cv='prefit', estimator=GaussianNB(), method='cumulated_score',
                random_state=1)

In the prediction step, we have to decide whether to include the last label or not. Let’s look at all 3 options,
starting with the default of including the label that crosses the threshold.

## Last label included

In the predict function, we need to specify what to do with the last label that crosses the threshold. In the following
code, we include it in the prediction set. The default is to include it, so we could have omitted the
'include_last_label' option.

In [7]:
y_pred, y_set = mapie_score.predict(X_new, alpha=0.05, include_last_label=True)
y_set = np.squeeze(y_set)

cov = classification_coverage_score(y_new, y_set)
setsize = classification_mean_width_score(y_set)
print('Coverage: {:.2%}'.format(cov))
print("Avg. set size: {:.2f}".format(setsize))
print(class_wise_performance(y_new, y_set, le.classes_))

Coverage: 96.96%
Avg. set size: 1.91
      class  coverage  avg. set size
0  BARBUNYA  0.931034       2.206897
1    BOMBAY  1.000000       1.000000
2      CALI  0.971591       2.153409
3  DERMASON  0.982759       1.512315
4     HOROZ  0.946721       1.979508
5     SEKER  0.982979       2.114894
6      SIRA  0.974277       2.131833


# Analysis

The coverage is slightly above 95%, which is to be expected, since by always including the label above the threshold, we
get a larger coverage.

The average set size is also larger than for the score method, which is also expected since the score method produces
the smallest sets on average.

For the class-wise coverage, we have to go into detail:
- The coverages are slightly better than for the score method, especially when looking at the lower bounds.
- Some coverages are well over 95%, because some classes are super easy to classify and always have a large score for
the true class; MAPIE would have to produce empty sets to artificially reduce the coverage, which would be stupid.

## Last label excluded
Let’s see what happens when we exclude the label.

In [ ]:
y_pred, y_set = mapie_score.predict(X_new, alpha=0.05, include_last_label=False)
y_set = np.squeeze(y_set)
cov = classification_coverage_score(y_new, y_set)
setsize = classification_mean_width_score(y_set)
print('Coverage: {:.2%}'.format(cov))
print("Avg. set size: {:.2f}".format(setsize))
print(class_wise_performance(y_new, y_set, le.classes_))

The set sizes are rather small, but this comes at a price: The coverage is much lower than the targeted 95%, which is
the result of excluding the label that would cross the threshold. In fact, this means that APS with excluded last label
is not really a conformal predictor, since it can’t guarantee anything.

<font color=red> Warning: Don’t use APS with last label excluded, because it ruins the coverage guarantee.
Use it only if you know what you are doing. </font>

## Include last label at random
Let’s try the last option and include the last label at random.

In [ ]:
y_pred, y_set = mapie_score.predict(X_new, alpha=0.05, include_last_label="randomized")
y_set = np.squeeze(y_set)
cov = classification_coverage_score(y_new, y_set)
setsize = classification_mean_width_score(y_set)
print('Coverage: {:.2%}'.format(cov))
print("Avg. set size: {:.2f}".format(setsize))
print(class_wise_performance(y_new, y_set, le.classes_))

Now the coverage is again close to the desired 95%. In fact, of all three inclusion options, the “randomized” option is
the only one that has the following guarantee:

$$
1-\alpha \le \mathbb{P} \left( Y_{test} \in C\left( X_{test} \right) \right) \le 1 - \alpha + \frac{1}{n_{cal} + 1}
$$

This coverage doesn’t hold for the other two options (include_last_label=True/False). The option ‘include_last_label=True’
can at least guarantee a coverage above $ 1-\alpha $.

There is another design choice of MAPIE to keep in mind: Of course, one would expect the coverage of the true label to
be lowest for “include_last_label=False”, increasing with “randomized”, and highest for “True”. In theory, this is
correct. But in practice, this doesn’t always happen. When setting include_last_label to True or False, MAPIE will not
produce empty prediction sets. But the “randomized” strategy can produce empty sets. This can lead to a situation where
both options (False/True) have a higher than nominal coverage of $ 1-\alpha $ and therefore higher than the “randomized” option.

## Conclusion: APS

APS can generate adaptive sets. If the randomization option is used, exact marginal coverage can be expected. When many
possible class labels are involved, APS can produce large prediction sets and RAPS may be a better option.

# Top-k method for fixed size sets

A simple alternative is the top-k method (Angelopoulos et al. 2020). The top-k method follows the same conformal
classification recipe as APS and the score method, but uses a different non-conformity score. Top-k uses only the rank
of the true class instead of the probability outcome. The higher the rank of the true class, the less certain the model
classification was. As usual, in the calibration step we find the threshold $\hat{q}$ for the score, in this case the rank.

Using the rank has a drastic effect on the prediction sets: they all have the same size (at least in theory).

In MAPIE it’s just a matter of changing the method parameter.

In [ ]:
mapie_score = MapieClassifier(model, cv="prefit", method="top_k")
mapie_score.fit(X_calib, y_calib)
y_pred, y_set = mapie_score.predict(X_new, alpha=0.05)
y_set = np.squeeze(y_set)
cov = classification_coverage_score(y_new, y_set)
setsize = classification_mean_width_score(y_set)
print('Coverage: {:.2%}'.format(cov))
print("Avg. set size: {:.2f}".format(setsize))
print(class_wise_performance(y_new, y_set, le.classes_))

The threshold in this case is 3 classes (<font color=red> 3 in the book, but 4 in our experiments </font>). Since the
top-k method can only cut at distinct points, the coverage will not be exactly $ 1 - \alpha $. It would be if we repeated
the experiment many times, because sometimes it might also cut at 4 or 2, and then on average it might reach $ 1 - \alpha $ coverage.

But there’s a problem: Why are some set sizes greater than 3? The answer is ties in probabilities. If a bean has the
following ordered probabilities: [0.80, 0.17, 0.01, 0.01, 0.01, 0, 0], then 5 classes are included in the set instead of 3,
because three classes are tied for third place.

## Conclusion: Top-k

Top-k is even simpler than the Score method and produces fixed size sets. This can be useful if you need to produce the
same set size for all data points. Top-k has the worst adaptivity as it literally produces the same size prediction sets
no matter how complicated the classification was.

# Regularized APS (RAPS) for small sets

The APS method tends to produce rather large prediction sets, especially if there are more than a handful of possible classes.
The reason: there can be a long tail of (noise) classes with low probability. RAPS fixes APS by introducing regularization.

## How it works

RAPS introduces a regularization term that penalizes the inclusion of too many classes. The regularization is based on
two new parameters, $\lambda$ and $k_{reg}$. We won’t go into the details of how the algorithm works, just the intuition
behind it.
- First, the class probabilities are sorted in descending order.
- All classes with a rank higher than $k_{reg}$ get a penalty term.
- This penalty depends on $\lambda$ and how many classes away the true label is from $k_{reg}$
- The penalty is added to the class probability.

Except for the regularization part, the procedure is the same as the APS procedure. The effect of the penalty is that
classes with low probabilities are less likely to be included, because the threshold $\hat{q}$ is reached earlier. The
regularization doesn’t come for free, since you have to sacrifice a percentage of the calibration data to tune the
regularization parameters.

## RAPS in MAPIE

As a user of MAPIE, you don’t have to worry about the regularization because it is done automatically.
By default, 20% of the calibration data is used for regularization.

In [ ]:
mapie_score = MapieClassifier(model, cv="prefit", method="raps")
mapie_score.fit(X_calib, y_calib, size_raps=0.2)
y_pred, y_set = mapie_score.predict(X_new, alpha=0.05, include_last_label="randomized")
y_set = np.squeeze(y_set)
cov = classification_coverage_score(y_new, y_set)
setsize = classification_mean_width_score(y_set)

print('Coverage: {:.2%}'.format(cov))
print("Avg. set size: {:.2f}".format(setsize))
print(class_wise_performance(y_new, y_set, le.classes_))

In this case, however, RAPS didn’t really reduce the average size of the prediction set. RAPS is more useful in cases
with many classes, such as ImageNet with thousands of classes.

## Conclusion: RAPS

RAPS produces smaller prediction sets than APS and is a compromise between Top-k (extreme regularization) and APS (no
regularization). In MAPIE, RAPS only works for already trained models (not for CV or LOO). The regularization requires
sacrificing some of the calibration data to tune the regularization parameters. RAPS is useful when you have many classes.

## Group-balanced conformal prediction

While methods like APS and RAPS can get you a little closer to conditional coverage, they by no means guarantee that
coverage will hold for every data point or subset of data points. But what if you have specific groups in the data for
which you want to guarantee coverage? Suppose you want the coverage guarantee for image classification to hold for both
animals and animals in costumes?

Fortunately, group-balanced conformal prediction is not only possible, it is easy and comes with guaranteed coverages
per group. Just divide the data into groups and perform the conformal prediction separately for each group. This
requires that you know the group at the time of prediction, so group-balanced CP won’t work if we split by class.

By doing group-balanced conformal prediction, we get the marginal coverage within each group. Sounds too good to be true!
But there is a price to pay: For group-wise CP, we have to split the calibration data by groups. If you have 1000 data
points in the calibration set and 10 groups of equal size, then you have only about 100 data points left for calibration
per group. As you increase the number of groups, and also if you have unbalanced groups, your calibration data per group
will quickly become small. Smaller calibration datasets mean less reliable threshold estimates. While it doesn’t hurt
the coverage guarantee (on average), there will be a lot more variance.

Let’s try this with the beans. Since all bean features are numeric, we use the area feature to define a group of small beans.

In [ ]:
X.Area.hist(bins = 100)
small_bean_index = (X_calib.Area < 70000)

Then we can simply apply CP to the group of small beans.

In [ ]:
X_calib_group = X_calib[small_bean_index]
y_calib_group = y_calib[small_bean_index]
mapie_group = MapieClassifier(model, cv="prefit", method="score")
mapie_group.fit(X_calib_group, y_calib_group.values)

For the prediction step, we need to apply the same group definition again to get a subset of the data, and then apply
the conformal predictor as usual.

In [ ]:
group_index = (X_new.Area < 70000)
X_group = X_new[group_index]
y_pred, y_set = mapie_group.predict(X_group, alpha=0.05)

We could do the same for the large beans. Then we get the marginal coverage guarantee for both groups. Of course, there
are more elegant coding solutions for iterating through the groups. This code just shows how group-balanced CP works in
principle.

## Conclusion: group-balanced CP
Group-balanced conformal prediction guarantees $ 1 - \alpha $ coverage for arbitrary groupings of the data. However, the
more groups there are, the fewer data points are left in the calibration set per group, and the calibration quality
suffers. And the group variable must also be available for new data.

## Class-Conditional APS (CCAPS) for coverage by class

Thinking back to the beans use case in the Getting Started chapter, the desired grouping would have been by bean variety.
But variety is the outcome to predict by the classification model, so we don’t have access to it at prediction time. If
we did, we wouldn’t need our machine learning model.

However, there’s still a way to guarantee class-wise coverage: class-conditional conformal prediction.

Class-conditional APS (CCAPS) is an approach to guarantee $ 1 - \alpha $ coverage of prediction sets per class. In many
use cases, this is a useful property to have, as it makes the conformal predictor much more adaptive and gives equal
attention to each class.

The calibration step is the same as for group-balanced conformal prediction, and we use class as the grouping variable.
Since the class is known for the calibration set, we can simply split our data by class and apply conformal prediction
as we normally would, but by class. For the beans example, we would end up with 7 conformal predictors.

The “problem” occurs in the prediction step: we don’t know the true classes for the new data. The solution
(Derhacobian et al.): We apply all resulting classwise conformal predictors, and the prediction set is the union of all
conformal classes.

Let’s say you have $ 𝑘 = 3 $ classes, you pick $ \alpha = 0.1 $, and the respective thresholds $\hat{q}_k$ are 0.95, 0.8,
and 0.99. Then CCAPS chooses $\hat{q} = 0.99$. This choice guarantees for all classes that we have a coverage of
$ \ge 1 - \alpha $, in our case 10%. However, this also means that for some classes the coverage can be much higher than
$ 1 - \alpha $ and thus produce large prediction sets. In this way, for at least one class is the expected coverage at
roughly $ 1 - \alpha $ and the others will automatically be larger, somewhere between 1 and $ 1 - \alpha $. In our
fictional case, they could be 92% for class 1, 98% for class 2, and 90% for class 3.

## Conclusion: class-conditional CP
Class conditional CP guarantees $ \ge 1 - \alpha $ coverage per class. Like group-wise CP, it reduces the calibration
performance by splitting the calibration data into many classes. Also, it can produce rather large prediction sets,
since it has to guarantee coverage for the most difficult class.

# Guide for choosing a conformal classification method

This chapter has covered many different conformal classification methods. Which one should you use when?

That depends mostly on how much you care about conditional coverage. I have ordered the recommendations from
"don’t care" to "it’s important".

- You need fixed size prediction sets ⇒ Use Top-k
- You don’t care about conditional coverage at all ⇒ Score method
- You don’t care about conditional coverage as long as coverage per group is guaranteed
    - If you have enough data per group ⇒ Group balanced conformal prediction
    - If you don’t have enough data ⇒ APS or RAPS
- You don’t care about conditional coverage as long as each class has guaranteed coverage ⇒ Class conditional conformal prediction
- You care about conditional coverage in general
    - Not too many classes ⇒ APS
    - You want extra small prediction sets and can sacrifice some of the calibration data, or you have not just a handful but hundreds or thousands of classes ⇒ RAPS